In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import LayerNormalization 
from tensorflow.keras import activations,callbacks
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model

from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.preprocessing import StandardScaler,scale
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import numpy as np 
import pandas as pd
import math
import random
import sys
import warnings
warnings.filterwarnings("ignore")

print(tf.__version__)
print(keras.__version__)

In [ ]:
!conda list

In [ ]:
!pip -V
# !pip show pandas

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv')

In [ ]:
train.drop(['id'], axis=1, inplace=True)
test.drop(['id'], axis=1, inplace=True)
y = np.array(train['loss'])
X = train.drop(['loss'], axis=1)
X_all = pd.concat([X, test], axis=0, copy=False).reset_index(drop=True)
y.shape, X.shape, test.shape, X_all.shape

In [ ]:
X_median = pd.DataFrame.median(X_all, 0)
X_25quan = X_all.quantile(0.25, 0)
X_75quan = X_all.quantile(0.75, 0)
X_all = (X_all - X_median) / (X_75quan - X_25quan)

In [ ]:
def quantile_normalization(df_input):
    sorted_df = pd.DataFrame(np.sort(df_input.values, axis=0)
                             , index=df_input.index
                             , columns=df_input.columns)
    mean_df = sorted_df.mean(axis=1)
    mean_df.index = np.arange(1, len(mean_df) + 1)
    quantile_df = df_input.rank(axis=0, method='min').stack().astype(int).map(mean_df).unstack()
    
    return (quantile_df)

quantile_all = np.array(quantile_normalization(X_all))
quantile_labeled = quantile_all[:len(train), :]
quantile_unlabeled = quantile_all[len(train):, :]

In [ ]:
bins_all = np.zeros((quantile_all.shape[0], X.shape[1]))
for i in range(X.shape[1]):
    bins_all[:, i] = pd.qcut(quantile_all[:, i],
                             X.shape[1], labels=False,
                             duplicates='drop')
bins_labeled = bins_all[:X.shape[0], :]
bins_unlabeled = bins_all[X.shape[0]:, :]

In [ ]:
noise = np.random.normal(0, .1, (quantile_all.shape[0], quantile_all.shape[1]))
quantile_all = np.array(quantile_all)
X_noisy = quantile_all + noise
limit = np.int(0.8 * quantile_all.shape[0])
X_train = X_noisy[0:limit, :]
y_train = quantile_all[0:limit,]
X_valid = X_noisy[limit:quantile_all.shape[0], :]
y_valid = quantile_all[limit:quantile_all.shape[0], :]
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=20,
    verbose=0,
    mode='min',
    baseline=None,
    restore_best_weights=True
)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.8,
    patience=4,
    verbose=0,
    mode='min'
)

In [ ]:
def custom_loss(y_true, y_pred):
    loss = K.mean(K.square(y_pred - y_true))
    return loss

In [ ]:
def autoencoder():
    ae_input = layers.Input(shape=(quantile_all.shape[1]))
    ae_encoded = layers.Dense(
        units=quantile_all.shape[1],
        activation='elu'
    )(ae_input)
    ae_encoded = layers.Dense(
        units=quantile_all.shape[1]*3,
        activation='elu'
    )(ae_encoded)
    ae_decoded = layers.Dense(
        units=quantile_all.shape[1],
        activation='elu'
    )(ae_encoded)
    
    return Model(ae_input, ae_decoded), Model(ae_input, ae_encoded)

In [ ]:
autoencoder, encoder = autoencoder()
autoencoder.compile(
    loss=custom_loss,
    optimizer=keras.optimizers.Adam(lr=5e-3)
)
history = autoencoder.fit(
    X_train,
    y_train,
    epochs=200,
    batch_size=512,
    verbose=0,
    validation_data=(X_valid, y_valid),
    callbacks=[early_stopping, plateau]
)

In [ ]:
encoded_all = encoder.predict(quantile_all)
print('Max encoded value is: ',np.max(encoded_all))

In [ ]:
encoded_variance = np.var(encoded_all, axis=0, ddof=1)
encoded_variance_1 = encoded_variance > 0.8
selected_cols = np.where(encoded_variance_1 == False, encoded_variance_1, 1)
num_cols = selected_cols.sum()
print('Number of selected columns is: ', num_cols)

if ((num_cols < 95) | (num_cols > 110)) == True:
    sys.exit()
    
encoded_all_1 = pd.DataFrame()
for i in range(quantile_all.shape[1]*3):
    if encoded_variance_1[i] == True:
        col_name = f'col_{i}'
        encoded_all_1[col_name] = encoded_all[:, i]
    
encoded_all_1 = np.array(encoded_all_1)
encoded_labeled = encoded_all_1[:len(train), :]
encoded_unlabeled = encoded_all_1[len(train):, :]
encoded_labeled.shape, encoded_unlabeled.shape

In [ ]:
pca = PCA(n_components=10)
pca_all = pca.fit_transform(encoded_all)
stand_scaler_pca = StandardScaler()
pca_all = stand_scaler_pca.fit_transform(pca_all)

In [ ]:
pca_labeled = pca_all[:len(train), :]
pca_unlabeled = pca_all[len(train):, :]
encoded_labeled = np.hstack((encoded_labeled, pca_labeled))
encoded_unlabeled = np.hstack((encoded_unlabeled, pca_unlabeled))

In [ ]:
def get_res_model():
    input_Q = layers.Input(shape = (quantile_all.shape[1]))
    input_E = layers.Input(shape = (encoded_labeled.shape[1]))
    input_B = layers.Input(shape = (bins_labeled.shape[1]))
    
    dense_QE = layers.Dropout(0.3)(layers.Concatenate()([input_Q, input_E]))
    dense_QE = tfa.layers.WeightNormalization(
        layers.Dense(
            units=300,
            activation='elu',
            kernel_initializer='lecun_normal'
        )
    )(dense_QE)
    
    embed_CB = layers.Embedding(
        input_dim=bins_labeled.shape[1]+1,
        output_dim=6,
        embeddings_regularizer='l2',
        embeddings_initializer='lecun_uniform'
    )(input_B)
    embed_CB = layers.Dropout(0.3)(embed_CB)
    embed_CB = layers.Conv1D(6, 1, activation='relu')(embed_CB)
    embed_CB = layers.Flatten()(embed_CB)
    
    hidden = layers.Dropout(0.3)(layers.Concatenate()([dense_QE, embed_CB]))
    hidden = tfa.layers.WeightNormalization(
        layers.Dense(
            units=64,
            activation='elu',
            kernel_initializer='lecun_normal'
        )
    )(hidden)
    
    output = layers.Dropout(0.3)(layers.Concatenate()([embed_CB, hidden]))
    output = tfa.layers.WeightNormalization(
        layers.Dense(
            units=32,
            activation='relu',
            kernel_initializer='lecun_normal'
        )
    )(output)
    
    output = layers.Dropout(0.4)(layers.Concatenate()([embed_CB, hidden, output]))
    output = tfa.layers.WeightNormalization(
        layers.Dense(
            units=32,
            activation='selu',
            kernel_initializer='lecun_normal'
        )
    )(output)
    output = layers.Dense(
        units=1,
        activation='selu',
        kernel_initializer='lecun_normal'
    )(output)
    
    model = Model([input_Q, input_E, input_B], output)
    model.compile(
        loss='mse',
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
        optimizer=keras.optimizers.Adam(lr=0.005)
    )
    
    return model

In [ ]:
print(np.__version__)
!python -V

In [ ]:
N_FOLDS = 10
SEED = 1
EPOCH = 100
N_ROUND = 5

oof = np.zeros((y.shape[0], 1))
pred = np.zeros((test.shape[0], 1))

for i in range(N_ROUND):
    oof_round = np.zeros((y.shape[0], 1))
    skf = StratifiedKFold(
        n_splits=N_FOLDS,
        shuffle=True,
        random_state = SEED*i
    )
    
    for fold, (tr_idx, ts_idx) in enumerate(skf.split(X, y)):
        print(f'\n------ TRAINING ROUND {i} FOLD {fold} ------\n')
        q_train = quantile_labeled[tr_idx]
        q_valid = quantile_labeled[ts_idx]
        
        b_train = bins_labeled[tr_idx]
        b_valid = bins_labeled[ts_idx]
        
        e_train = encoded_labeled[tr_idx]
        e_valid = encoded_labeled[ts_idx]
        
        y_train = y[tr_idx]
        y_valid = y[ts_idx]
        
        K.clear_session()
        
        model = get_res_model()
        model.fit(
            [q_train, e_train, b_train],
            y_train,
            batch_size=2048,
            epochs=EPOCH,
            validation_data=([q_valid, e_valid, b_valid], y_valid),
            callbacks=[early_stopping, plateau],
            verbose=0
        )
        
        pred_round = model.predict([q_valid, e_valid, b_valid])
        oof[ts_idx] += pred_round / N_ROUND
        oof_round[ts_idx] += pred_round
        pred += model.predict([quantile_unlabeled, encoded_unlabeled, bins_unlabeled]) / (N_FOLDS * N_ROUND)
        
    score_round = math.sqrt(mean_squared_error(y, oof_round))
    print(f'===== SCORE ROUND (i): {score_round} =====\n')

score_round = math.sqrt(mean_squared_error(y, oof))
print(f'\n***** FINAL SCORE MODEL: (score_round) ******\n')
    

In [ ]:
sample_submission = pd.read_csv('../input/tabular-playground-series-aug-2021/sample_submission.csv')
sample_submission['loss'] = pred
pd.DataFrame(oof).to_csv('oof_2.csv', index=False)
sample_submission.to_csv('submission_v2.csv', index=False)
display(pd.read_csv('submission_v2.csv'))